In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('finalproject')\
        .config('spark.driver.extraClassPath','/usr/lib/jvm/java-17-openjdk-amd64/lib/postgresql-42.6.0.jar')\
        .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/09/12 11:33:48 WARN Utils: Your hostname, SUSHAN resolves to a loopback address: 127.0.1.1; using 172.22.231.246 instead (on interface eth0)
23/09/12 11:33:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/12 11:33:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
hourlyPrices_df = spark.read.format('jdbc').options(url="jdbc:postgresql://localhost:5432/sparkProject", driver = 'org.postgresql.Driver', dbtable='hourly_gasoline_prices', user='sushan',password='7446').load()
stationInfo_df = spark.read.format('jdbc').options(url="jdbc:postgresql://localhost:5432/sparkProject", driver = 'org.postgresql.Driver', dbtable='fuel_station_information', user='sushan',password='7446').load()

In [4]:
countt = hourlyPrices_df.count()
print("Total number of records in hourlyPrices:", countt)
counttt = stationInfo_df.count()
print("Total number of records in stationInfo:", counttt)

Total number of records in hourlyPrices: 2444337
Total number of records in stationInfo: 22129


In [5]:
hourlyPrices_df.show()

+-----+------+-----+-------------------+
|   Id|isSelf|Price|               Date|
+-----+------+-----+-------------------+
|43109|     0|1.989|2022-01-04 05:43:06|
|30524|     0|2.059|2022-01-04 06:12:26|
|23359|     1|1.799|2022-01-04 06:50:35|
|53189|     1|1.739|2022-01-04 06:57:35|
|13242|     1|1.749|2022-01-04 07:55:48|
|48977|     0| 1.74|2022-01-04 08:00:50|
|25216|     0|1.889|2022-01-04 08:43:58|
|37535|     1|1.769|2022-01-04 08:48:30|
|50536|     0|2.061|2022-01-04 09:30:35|
|16536|     0|2.024|2022-01-04 10:17:24|
|11987|     0|1.829|2022-01-04 10:38:22|
| 3988|     0|2.309|2022-01-04 11:46:58|
|25561|     1|1.778|2022-01-04 12:43:43|
|53223|     0|1.799|2022-01-04 13:11:21|
|16791|     1|1.829|2022-01-04 13:15:27|
|46153|     1|1.779|2022-01-04 13:26:09|
|42357|     1|1.794|2022-01-04 13:26:42|
|22669|     1|1.809|2022-01-04 13:32:44|
|50036|     1|2.109|2022-01-04 13:35:54|
| 5243|     1|1.789|2022-01-04 13:42:13|
+-----+------+-----+-------------------+
only showing top

In [6]:
stationInfo_df.show()

+-----+--------------------+-------------------+------------+--------------------+--------------------+---------+-----------+
|   Id|Fuel_station_manager|     Petrol_company|        Type|        Station_name|                City| Latitude|Longitudine|
+-----+--------------------+-------------------+------------+--------------------+--------------------+---------+-----------+
|36419|GP GREEN POWER EX Q8|      Pompe Bianche|    Stradale|GP GREEN POWER EX Q8|           SICULIANA|37.336323|  13.425704|
|40677|Q8 viale della Vi...|                 Q8|    Stradale|Q8 viale della Vi...|              ANCONA|  43.6162|  13.521665|
|49102|street bar srl fa...|           Agip Eni|    Stradale|street bar srl fa...|            FABRIANO| 43.34446|  12.975039|
|47425|   PENTA PETROLI SRL|      Pompe Bianche|    Stradale|     EWA POINT TUFO |                TUFO|41.009964|  14.823013|
|23376|SERVIZI & GESTION...|                 Q8|    Stradale| ALTAMURA VIA CORATO|            ALTAMURA|40.835793|   16

In [7]:
print(hourlyPrices_df.printSchema(), stationInfo_df.printSchema())

root
 |-- Id: integer (nullable = true)
 |-- isSelf: integer (nullable = true)
 |-- Price: double (nullable = true)
 |-- Date: timestamp (nullable = true)

root
 |-- Id: string (nullable = true)
 |-- Fuel_station_manager: string (nullable = true)
 |-- Petrol_company: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Station_name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Latitude: float (nullable = true)
 |-- Longitudine: float (nullable = true)

None None


In [8]:
from pyspark.sql.functions import col, lag, avg, abs, lit, min, udf, dense_rank
from pyspark.sql.window import Window
from haversine import haversine, Unit
from pyspark.sql.types import FloatType

### Q1. Find the `Most Stable Company` and `Most Volatile Company` based on `Average Daily Price Variation`


In [9]:
df_forAvg = hourlyPrices_df.join(stationInfo_df.select("Id", "Petrol_company"), "Id")

window_spec = Window.partitionBy("Petrol_company").orderBy("Date")

daily_var = df_forAvg.withColumn("Previous_Price", lag("Price").over(window_spec))
daily_var = daily_var.withColumn("Price_Variation", abs(col("Price") - col("Previous_Price")))
# daily_var.show()

In [10]:
daily_var = daily_var.filter(col("Previous_Price").isNotNull())
# daily_var.show()

In [11]:
avg_variation_df = daily_var.groupBy("Petrol_company").agg(avg("Price_Variation").alias("Avg_Daily_Variation"))
# avg_variation_df.show()

In [12]:
min_variation_company = avg_variation_df.orderBy("Avg_Daily_Variation").first()
max_variation_company = avg_variation_df.orderBy(avg_variation_df["Avg_Daily_Variation"].desc()).first()

print("Most Stable Company:")
print("Company:", min_variation_company["Petrol_company"])
print("Average Daily Price Variation:", min_variation_company["Avg_Daily_Variation"])

print("_*"*33)

print("Most Volatile Company:")
print("Company:", max_variation_company["Petrol_company"])
print("Average Daily Price Variation:", max_variation_company["Avg_Daily_Variation"])


Most Stable Company:
Company: Blanco petroli
Average Daily Price Variation: 0.008190127970749542
_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*_*
Most Volatile Company:
Company: COLAGROSSI CARBURANTI
Average Daily Price Variation: 0.2854666666666667


### Q2. `Find 9 nearest stations to a certain reference station and calculate the price difference between the fuel station and its nearest competitor`

In [13]:
@udf(FloatType())
def calculate_distance(lat1, lon1, lat2, lon2):
    return haversine((lat1, lon1), (lat2, lon2), unit=Unit.KILOMETERS)

In [14]:
ref_latitude = 40.7160385
ref_longitude = 14.9413282

In [15]:
stationInfo_df = stationInfo_df.filter((col("Latitude").isNotNull()) & (col("Longitudine").isNotNull()))

In [16]:
dis_stationInfo_df = stationInfo_df.withColumn(
    "distance",
    calculate_distance(
        lit(ref_latitude),
        lit(ref_longitude),
        col("Latitude"),
        col("Longitudine")
    )
)

In [17]:
min_distance_df = dis_stationInfo_df.groupBy("Id").agg(min("distance").alias("min_distance"))
# min_distance_df.show()

In [18]:
price_diff_df = hourlyPrices_df.join(min_distance_df, "Id", "inner")
# price_diff_df.show()

In [19]:
window_spec = Window.orderBy(col("min_distance"))
ranked_stations_df = price_diff_df.withColumn("rank", dense_rank().over(window_spec))
deduplicated_stations_df = ranked_stations_df.dropDuplicates(['Id']).drop("Date")
filtered_stations_df = deduplicated_stations_df.filter(col("rank") <= 9).orderBy("rank")
filtered_stations_df = filtered_stations_df.drop('rank')
# filtered_stations_df.show()

In [20]:
window_spec = Window.orderBy("min_distance")
filtered_stations_df = filtered_stations_df.withColumn("Price_of_Nearest_Competitor", lag("Price").over(window_spec))
filtered_stations_df = filtered_stations_df.withColumn(
    "PriceDifferenceFromNearestCompetitor",
    col("Price") - col("Price_of_Nearest_Competitor")
)
filtered_stations_df.show()

23/09/12 11:36:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 11:36:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 11:36:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 11:36:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 11:36:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 11:36:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/12 1

+-----+------+-----+------------+---------------------------+------------------------------------+
|   Id|isSelf|Price|min_distance|Price_of_Nearest_Competitor|PriceDifferenceFromNearestCompetitor|
+-----+------+-----+------------+---------------------------+------------------------------------+
|38792|     1| 1.82|  0.22640005|                       null|                                null|
|52830|     1|1.929|   0.3241125|                       1.82|                 0.10899999999999999|
|17152|     0|2.014|    2.730627|                      1.929|                 0.08499999999999974|
|17409|     0|1.979|    2.848063|                      2.014|                 -0.0349999999999997|
|51180|     1|1.759|   2.9082253|                      1.979|                 -0.2200000000000002|
|27109|     1|1.709|   4.2238717|                      1.759|                -0.04999999999999982|
|46547|     1|1.799|    4.730566|                      1.709|                 0.08999999999999986|
|23047|   

In [21]:
# Stop the SparkSession
spark.stop()